In [ ]:
! pip install evaluate

In [ ]:
from datasets import load_dataset
emotions = load_dataset("emotion")
emotions

### Load tokenizer

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True)

emotion_encoded = emotions.map(tokenize, batched=True)

In [ ]:
emotion_encoded = emotion_encoded.remove_columns(["text"])
emotion_encoded = emotion_encoded.rename_column("label", "labels")
emotion_encoded.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

In [ ]:
train_data = emotion_encoded["train"]
eval_data = emotion_encoded["test"]

### DataLoader

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_data, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(eval_data, batch_size=8)

### Load model

In [ ]:
from transformers import AutoModelForSequenceClassification
num_labels = 6
model_ckpt = "distilbert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels) # with classification head 

### Opitimizer

In [ ]:
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)

### Learning rate scheduler

In [ ]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

### Training loop

In [ ]:
from tqdm.auto import tqdm
progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k:v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

In [ ]:
import evaluate

metric = evaluate.load("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()